# Test Model Recurrant Neural Network

In [1]:
# Step 1: Clone the GitHub repository
!git clone https://github.com/Muhammad-Ikhwan-Fathulloh/Hands-On-NLP-Super-Class-Batch1.git

Cloning into 'Hands-On-NLP-Super-Class-Batch1'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 115 (delta 36), reused 92 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 2.88 MiB | 9.06 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [2]:
# Step 2: Import necessary libraries
import torch
import torch.nn as nn
import string
import unicodedata
import os

In [3]:
# Step 3: Set up model configurations
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Define the path to the dataset in the cloned repo
data_path = 'Hands-On-NLP-Super-Class-Batch1/RNN/data/names'
all_categories = [os.path.splitext(filename)[0] for filename in os.listdir(data_path) if filename.endswith('.txt')]
n_categories = len(all_categories)

In [4]:
# Define the RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        hidden = torch.tanh(self.i2h(input) + self.h2h(hidden))
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [5]:
# Step 4: Load the trained model from the cloned repo
rnn = RNN(n_letters, 128, n_categories)  # Use the hidden_size from training
rnn.load_state_dict(torch.load("Hands-On-NLP-Super-Class-Batch1/RNN/models/rnn.pt"))
rnn.eval()

<ipython-input-5-03e2b7cdaa43>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rnn.load_state_dict(torch.load("Hands-On-NLP-Super-Class-Batch1/RNN/models/rnn.pt"))


RNN(
  (i2h): Linear(in_features=57, out_features=128, bias=True)
  (h2h): Linear(in_features=128, out_features=128, bias=True)
  (h2o): Linear(in_features=128, out_features=18, bias=True)
  (softmax): LogSoftmax(dim=1)
)

In [6]:
# Utility functions
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i]

def evaluate(line_tensor):
    hidden = rnn.initHidden()
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    return output

In [7]:
# Prediction function
def predict(name):
    input_name = unicodeToAscii(name)
    input_tensor = lineToTensor(input_name)
    with torch.no_grad():
        output = evaluate(input_tensor)
        category = categoryFromOutput(output)
        confidence = torch.exp(output.max()).item()
    return {"category": category, "confidence": confidence}

In [21]:
# Dynamic input from user
name = input("Enter a name: ")
prediction = predict(name)
print(f"Predicted category: {prediction['category']}")
print(f"Confidence: {prediction['confidence']}")

Enter a name: ikhwan
Predicted category: Russian
Confidence: 0.3670042157173157
